In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, 5)

# kafkaStream = KafkaUtils.createStream(ssc, 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092', 'spark-streaming-consumer', {'mo-topic':1})  

In [2]:
directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092'})

In [3]:
lines = directKafkaStream.map(lambda x: x[1])
# counts = lines.flatMap(lambda line: line.split(" ")) \
#     .map(lambda word: ("word_count", 1)) \
#     .reduceByKey(lambda a, b: a+b)
# counts.pprint()

In [4]:
import re

In [5]:
# counts = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
#     .map(lambda word: ("line_count", 1)) \
#     .reduceByKey(lambda a, b: a+b)
# counts.pprint()



In [6]:
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0

In [7]:
clicks = lines.flatMap(lambda line: [line.split("\t")[-1]]) \
    .map(lambda click: ("click_count", mk_int(click))) \
    .reduceByKey(lambda a, b: a+b)
clicks.pprint()

In [8]:
ssc.start()  
ssc.awaitTermination()




-------------------------------------------
Time: 2017-09-18 04:26:05
-------------------------------------------
('click_count', 56)

-------------------------------------------
Time: 2017-09-18 04:26:10
-------------------------------------------
('click_count', 80)

-------------------------------------------
Time: 2017-09-18 04:26:15
-------------------------------------------
('click_count', 65)

-------------------------------------------
Time: 2017-09-18 04:26:20
-------------------------------------------
('click_count', 78)

-------------------------------------------
Time: 2017-09-18 04:26:25
-------------------------------------------
('click_count', 76)



KeyboardInterrupt: 

In [ ]:
ssc.stop()
sc.stop()